# Create and run a pipeline in AML studio
- [docs MS reference](https://docs.microsoft.com/en-us/azure/machine-learning/concept-ml-pipelines)
- [mslearn-dp100 tutorial: 08 Create a Pipeline](https://github.com/MicrosoftLearning/mslearn-dp100/blob/main/08%20-%20Create%20a%20Pipeline.ipynb)
- [another repo example](https://github.com/liupeirong/MLOpsManufacturing/blob/7e61b72ed93012472fe453303659c9f2c35c880d/samples/image-classification-tensorflow/ml_service/pipelines/build_data_processing_pipeline.py)
- [About the Dataset class](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.dataset.dataset?view=azure-ml-py)


# Paddy's Suggestion
## on the naming convention of our code(s)
- vegetation density scenario modelling (veg forecasting) 
- (poc1) vegetation management tool (segmentation) 
## Documentation
- start recording the computing performance when scaling up
## Code restructuring 
- incorporate parameters from the .env file in the root folder

In [5]:
# A simple pipeline structure that can run different steps in a compute cluster
import azure.core
from azureml.core import Workspace, Datastore, Environment, Experiment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline
from azureml.data import OutputFileDatasetConfig


In [7]:
ws = Workspace.from_config()
# print(ws)

In [8]:
ws = Workspace.from_config()
# print(ws)
blob_store = Datastore(ws, "workspaceblobstore")    # workspaceblobstore is the default

# Create Experiment
experiment_name = 'vdsm_fusion_test1'
exp = Experiment(ws, experiment_name)

# Create Compute Target
aml_compute_target = 'train-veg'  # ALL SMALL LETTER, NO underscore, 16ws long only.
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("This Compute Target already exist.")
except ComputeTargetException:
    print("creating new compute target :", aml_compute_target)
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                                min_nodes=1, 
                                                                max_nodes=4,
                                               idle_seconds_before_scaledown=3000)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
print("Azure Machine Learning Compute Cluster attached now")

# Create environment to execute your code
env_name = "vdsm_env_all_test1"   #TBD
# env = Environment.from_conda_specification(name=env_name, file_path="./aml_test_include_all.yml")  
# Note that it takes 30min to install all the packages in *all.yml. Recommend install once and add on.

envs = Environment.list(workspace=ws)
if env_name in envs:
    # Use an existing environment
    env = Environment.get(workspace=ws, name=env_name)
else:
    # Create a new env using yml
    env = Environment.from_conda_specification(name=env_name, file_path="./aml_test_include_all.yml")

# ### Here are some examples about how to add packages to an existing env.
# conda_dep = CondaDependencies()
# # Installs numpy version 1.17.0 conda package
# conda_dep.add_conda_package("numpy==1.17.0")
# # Installs pillow package
# conda_dep.add_pip_package("pillow")
# # Adds dependencies to PythonSection of myenv
# env.python.conda_dependencies=conda_dep


This Compute Target already exist.
Azure Machine Learning Compute Cluster attached now


In [ ]:
## Pipeline blueprint: TBD
# TBD
### this is from an example here https://docs.microsoft.com/en-us/azure/machine-learning/concept-ml-pipelines
input_data = Dataset.File.from_files(
    DataPath(datastore, 'xyz/example.pkl'))

prepped_data_path = OutputFileDatasetConfig(name="output_path")

###  Catriona task: this *.py will take in the monthly data for a given area and concat them in time
dataprep_step = PythonScriptStep(
    name="prep_data_for_fusion",
    script_name="dataprep_fusion.py",
    source_directory="prep_src",
    compute_target=aml_compute_target,
    arguments=["--prepped_data_path", prepped_data_path],
    inputs=[input_dataset.as_named_input('raw_data').as_mount() ]
    )

prepped_data = prepped_data_path.read_delimited_files()

### Tiantian task: convert core data fusion script into veg_data_fusion.py
### Tiantian + Catriona: think about how to pass on multiple datasets in veg_data_fusion from here? 
### Tiantian: how to refactor the .py to accept arguments about the file location in blobs using keyvaults

s2_path = None   # ToDo: figure out how to pass on a blob location
fc_path = None

# Example that works in the original code
# co2_path = create_blob_stream('odc_vm_data/odc_outputs/co2_mm_gl_url.txt', t_expire=8)
co2_path = None

#  Create a url stream for the monthly climate data
# climate_path = create_blob_stream('/home/azureuser/cloudfiles/code/Users/Tiantian.Yuan/test_only/era-mon-2015-2021.nc', t_expire=8)
climate_path = None
mask_path = None

fusion_step = PythonScriptStep(
    name="veg_data_fusion",
    script_name="vdsm_traindata_fusion.py",
    compute_target=aml_compute_target,
    arguments=[
        "--s2_path", s2_path,
        "--fc_path", fc_path,
        "--co2_path", co2_path,
        "--climate_path", climate_path,
        "--mas_path", mask_path
    ],
    source_directory="prep_src"
)

### Catriona task: concat the output from the last step from different areas and produce a master tabular data. 
###   possible add registering the master dataset to our ML DevOps pipeline step.
concat_step = PythonScriptStep(
    name="VDSM_concat",
    script_name="VDSM_concat.py",
    compute_target=aml_compute_target,
    arguments=["--prepped_data", prepped_data],
    source_directory="prep_src"
)

steps = [dataprep_step, fusion_step, concat_step]

pipeline = Pipeline(workspace=ws, steps=steps)
pipeline_run = exp.submit(pipeline)
pipeline_run.wait_for_completion()


In [ ]:
## next step, publish the pipeline and obtain an endpoint

## Nov 29. 2021, priority for Catriona: testing Azure Function's calling  the endpoint after publishing a mock pipeline, e.g. follow this example
## https://github.com/MicrosoftLearning/mslearn-dp100/blob/main/08%20-%20Create%20a%20Pipeline.ipynb

- Dec 1. 2021. update This priority is done. Thanks Catriona! 